# web of science 学科共现

In [2]:
import os
import pandas as pd
import numpy as np
import networkx as nx
#import matplotlib.pyplot as plt
os.chdir('D:/software/0install package/VOSviewer/WOS_co_occurrence0304')

In [4]:
#data = pd.read_excel("D:/software/0install package/VOSviewer/web tab.xlsx")
# data = pd.read_table("help/a.txt",sep="	",index_col=False,keep_default_na=True)keep_default_na=True会让他
# df_wc = data[data.WC.fillna('').str.contains('; ')] 
data = pd.read_table("WOS tab0304_utf8.txt",sep="	",index_col=False)
df_wc = data[data.WC.str.contains('; ')] #筛选包含分隔符的数据，web of science是分号+空格,只有有分割符才代表是出现了两个及以上学科
print(len(data))
print(len(df_wc))#对比两个数字查看比例，大部分论文(接近50%)只有一个学科，所以学科共现不一定有意义

254
123


In [5]:
xueke_list = []
for wc in df_wc.WC:
    wc_list = wc.split("; ")
    xueke_list.extend(wc_list)
xueke_list_unique = list(set(xueke_list))
print(len(xueke_list_unique))

80


In [6]:
xueke_list_unique#web of science上边导出的学科包含二级学科，如果想用一级学科的话需要替换学科名，把逗号后边的二级删掉

['Mathematics, Interdisciplinary Applications',
 'Geriatrics & Gerontology',
 'Health Policy & Services',
 'Substance Abuse',
 'Mathematical & Computational Biology',
 'Energy & Fuels',
 'Computer Science, Theory & Methods',
 'Mathematics, Applied',
 'Otorhinolaryngology',
 'Engineering, Multidisciplinary',
 'Education, Scientific Disciplines',
 'Imaging Science & Photographic Technology',
 'Chemistry, Physical',
 'Construction & Building Technology',
 'Chemistry, Analytical',
 'Agriculture, Multidisciplinary',
 'Management',
 'Computer Science, Interdisciplinary Applications',
 'Computer Science, Cybernetics',
 'Cell Biology',
 'Pharmacology & Pharmacy',
 'Education, Special',
 'Peripheral Vascular Disease',
 'Oceanography',
 'Statistics & Probability',
 'Geosciences, Multidisciplinary',
 'Physics, Applied',
 'Psychology, Multidisciplinary',
 'Optics',
 'Education & Educational Research',
 'Computer Science, Software Engineering',
 'Meteorology & Atmospheric Sciences',
 'Physics, Math

In [9]:
xueke_frame = pd.DataFrame(np.zeros([len(xueke_list_unique),len(xueke_list_unique)]),columns=xueke_list_unique,index=xueke_list_unique)
for wc in df_wc.WC:
    wc_list = wc.split('; ')
    for i in range(len(wc_list)-1):
        #xueke_frame.loc[wc_list[i]][wc_list[i+1]]=xueke_frame.loc[wc_list[i]][wc_list[i+1]]+1
        for j in range(i+1,len(wc_list)-1):
            xueke_frame.loc[wc_list[i]][wc_list[j]]=xueke_frame.loc[wc_list[i]][wc_list[j]]+1

for k in range(len(xueke_frame)):#对角线都为0
    xueke_frame.iloc[k,k]=0

In [10]:
g = nx.Graph()
for i in range(len(xueke_list_unique)-1):
    xueke = xueke_list_unique[i]
    for j in range(i+1,len(xueke_list_unique)):
        w=0
        xueke2 = xueke_list_unique[j]
        w = xueke_frame.loc[xueke][xueke2]+xueke_frame.loc[xueke2][xueke]
        g.add_edge(xueke,xueke2,weight=w)
nx.write_pajek(g,"D:/software/0install package/VOSviewer/web tab.net")


In [ ]:
# import matplotlib.pyplot as plt
# nx.draw(g)
# plt.show()
#nx.write_pajek(g,"D:/software/0install package/VOSviewer/web tab.net")
#将net文件导入vosviewer

# 中国知网-共现

In [1]:
import os
import pandas as pd
import numpy as np
import networkx as nx
#import matplotlib.pyplot as plt
os.chdir('D:/work/网络课程开发')

In [2]:
data = pd.read_excel("CNKI.xlsx")
data.columns

Index(['SrcDatabase-来源库', 'Title-题名', 'Author-作者', 'Organ-单位', 'Source-文献来源',
       'Keyword-关键词', 'Year-年'],
      dtype='object')

In [3]:
data.columns = (['SrcDatabase', 'Title', 'Author', 'Organ', 'Source','Keyword','Year'])

In [8]:
organ_list = []
for organ in data.Keyword:
    organlist = organ.strip(';').split(";")#作者和机构
    #organlist = organ.split(";;")#关键词
    organ_list.extend(organlist)
organ_list_unique = list(set(organ_list))
print(len(organ_list_unique))

1334


In [9]:
organ_frame = pd.DataFrame(np.zeros([len(organ_list_unique),len(organ_list_unique)]),columns=organ_list_unique,index=organ_list_unique)
for organ in data.Keyword:
    organlist = organ.strip(';').split(";")#作者和机构
    #organlist = organ.split(";;")#关键词
    if len(organlist)>1:
        for i in range(len(organlist)-1):
            for j in range(i+1,len(organlist)-1):
                organ_frame.loc[organlist[i]][organlist[j]]=organ_frame.loc[organlist[i]][organlist[j]]+1

for k in range(len(organ_frame)):#对角线都为0
    organ_frame.iloc[k,k]=0

In [10]:
g = nx.Graph()
for i in range(len(organ_list_unique)-1):
    organ = organ_list_unique[i]
    for j in range(i+1,len(organ_list_unique)):
        w=0
        organ2 = organ_list_unique[j]
        w = organ_frame.loc[organ][organ2]+organ_frame.loc[organ2][organ]
        if w>0:
            g.add_edge(organ,organ2,weight=w)

In [11]:
nx.write_pajek(g,"cnki_keyword.net")
#将net文件导入vosviewer